# Modeling Phase – Baseline & Classical Models

In this notebook, we:

- Implement naive and moving average baseline forecasts to establish reference performance.
- Build and evaluate ARIMA/SARIMA models on the target energy consumption time series.
- Compare results using standard metrics (MAE, RMSE, MAPE, R²).


# Libraries

In [1]:
import pickle

## Loading cleaned data
Loading the cleaned data from EDA phase

In [3]:
with open('../data/data_cleaned.pkl', 'rb') as f:
    data_cleaned = pickle.load(f)

df_data = data_cleaned['df_data']
df_data_24h_complete = data_cleaned['df_data_24h_complete']

df_data.head()

,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday,EnergyConsumption,Date,Time,DayOfWeek_ext
Timestamp,,,,,,,,,,,,,
2022-01-01 00:00:00,25.139433,43.431581,1565.693999,5,On,Off,2.774699,Monday,No,75.364373,2022-01-01,0,Saturday
2022-01-01 01:00:00,27.731651,54.225919,1411.064918,1,On,On,21.831384,Saturday,No,83.401855,2022-01-01,1,Saturday
2022-01-01 02:00:00,28.704277,58.907658,1755.715009,2,Off,Off,6.764672,Sunday,No,78.270888,2022-01-01,2,Saturday
2022-01-01 03:00:00,20.080469,50.371637,1452.316318,1,Off,On,8.623447,Wednesday,No,56.519850,2022-01-01,3,Saturday
2022-01-01 04:00:00,23.097359,51.401421,1094.130359,9,On,Off,3.071969,Friday,No,70.811732,2022-01-01,4,Saturday


# Baseline model

# ARIMA model

# SARIMA model